<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/CNN-inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Conv1D, Dense, AveragePooling1D
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [4]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv', chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [7]:
model = Sequential()
model.add(Conv1D(activation="sigmoid", input_shape=(len(test.columns)-1, 1), kernel_size=5, filters=12))
model.add(AveragePooling1D(pool_size=2))
model.add(Conv1D(activation="sigmoid", kernel_size=5, filters=6))
model.add(AveragePooling1D(pool_size=2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 69, 100)           300       
                                                                 
 average_pooling1d_1 (Averag  (None, 34, 100)          0         
 ePooling1D)                                                     
                                                                 
 dense_1 (Dense)             (None, 34, 1)             101       
                                                                 
Total params: 401
Trainable params: 401
Non-trainable params: 0
_________________________________________________________________


In [8]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x, train_y, batch_size=32)

ValueError: ignored

In [ ]:
test_y = test.pop('Label')
test_x = test

In [ ]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [ ]:
pred = model.predict(test_x, verbose=1)
pred_y = np.round(pred).astype(int)

In [ ]:
confusion_matrix(test_y, pred_y)

In [ ]:
report = classification_report(test_y, pred_y, digits=5)
print(report)